<a href="https://colab.research.google.com/github/midhubalan/dojolicious-bot/blob/main/hello_claude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anthropic

In [ ]:
from functools import partial
from typing import Literal

from google.colab import userdata

from anthropic import (
    Anthropic,
    # AsyncAnthropic,
)

In [ ]:
client = Anthropic(api_key=userdata.get('ANTHROPIC_API_KEY'))
model = "claude-sonnet-4-0"

In [ ]:
def append_message(messages, *, role: str,  content: str):
  messages.append({"role": role, "content": content})
  return messages

add_user_message = partial(append_message, role='user')
add_assistant_message = partial(append_message, role='assistant')

def chat(messages, system=None):
  params = {
    "model":model,
    "max_tokens":1024,
    "messages": messages,
    # stream=True,
  }
  if system:
    params["system"] = system
  message = client.messages.create( **params
    )
  return message.content[0].text

def chat_stream(messages, system=None):
  params = {
    "model":model,
    "max_tokens":1024,
    "messages": messages,
    "stream": True,
  }
  if system:
    params["system"] = system
  with client.messages.create(**params) as stream:
    for text in stream.text_stream():


In [ ]:
messages = []
add_user_message(messages, content="Define quantum computing in one sentence")

resp = chat(messages)
print(resp)

add_assistant_message(messages, content=resp)

add_user_message(messages, content="Write another sentence")

add_assistant_message(messages, content=chat(messages))

In [ ]:
for message in messages:
  print(message)

In [ ]:
system = """
You are a patient math tutor.
Do not directly answer a student's questions.
Guide them to a solution step by step.
"""
messages = []
add_user_message(messages, content="How do I solve 5x + 2 = 3 for x?")
add_assistant_message(messages, content=chat(messages))
for message in messages:
  print(message)
print("****" * 5)
messages = []
add_user_message(messages, content="How do I solve 5x + 2 = 3 for x?")
add_assistant_message(messages, content=chat(messages, system))
for message in messages:
  print(message)

In [ ]:
messages = []
user_message= """
Write a python function that checks a string for duplicate characters.
"""
add_user_message(messages, content=user_message)
system = """
  You're a helpful coding assistant. When asked for help just provide the code that solves the problem. Do not explain the code, unless you are explicitly asked to do so by the user.
"""
add_assistant_message(messages, content=chat(messages, system))
for message in messages:
  print(message)

print()
messages = []
user_message += "Explain the code in two sentences."
add_user_message(messages, content=user_message)
add_assistant_message(messages, content=chat(messages, system))
for message in messages:
  print(message)

In [ ]:
with client.messages.stream(
    model=model,
    max_tokens=1024,
    stop_sequences=["```"],
    messages=[
        {
            "role": "user",
            "content": "Write a three different gcloud cli commands. Each should be very short"
        },
                {
            "role": "assistant",
            "content": "Here are all three commands with no comments in a single code block:\n```bash"
        },
    ],
) as stream :
  for text in stream.text_stream:
    print(text, end="", flush=True)
